## Retrieval Augmented FuctionCallingAgent


In this tutorial, we demonstrate how to utilize our `FunctionCallingAgent `implementation alongside a tool retriever.

The goal is to construct an agent that leverages the function-calling API of LLMs (such as OpenAI, Anthropic, or Mistral) to manage and index an arbitrary number of tools.

Our indexing and retrieval modules are designed to simplify the process, addressing the challenge of accommodating numerous functions within a prompt.

`FunctionCallingAgent` uses LLM function calling capabilities to complete the task.

In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

/Users/omprakash/lama-index/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/omprakash/lama-index/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

In [3]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers with and returns the result integer"""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two integers with and returns the result integer"""
    return a - b

def useless(a: int, b: int) -> int:
    """Toy useless function."""
    pass

In [4]:
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)

useless_tools = [
    FunctionTool.from_defaults(fn=useless, name=f"useless_{str(idx)}")
    for idx in range(50)
]

In [6]:
all_tools = [multiply_tool] + [add_tool] + [subtract_tool] + useless_tools

In [8]:
# define an "object" index over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [9]:
llm = OpenAI(model="gpt-3.5-turbo")

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=2),
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)
agent = AgentRunner(agent_worker)

In [10]:
response = agent.chat("What's 500 multiplied by 10?")

Added user message to memory: What's 500 multiplied by 10?
=== Calling Function ===
Calling function: multiply with args: {"a": 500, "b": 10}
=== Function Output ===
5000
=== LLM Response ===
500 multiplied by 10 is 5000.


In [11]:
print(response)

assistant: 500 multiplied by 10 is 5000.


In [12]:
response = agent.chat("What's 500 multiplied by 10 and then subtract 1000?")

Added user message to memory: What's 500 multiplied by 10 and then subtract 1000?
=== Calling Function ===
Calling function: multiply with args: {"a": 500, "b": 10}
=== Function Output ===
5000
=== Calling Function ===
Calling function: subtract with args: {"a": 5000, "b": 1000}
=== Function Output ===
4000
=== LLM Response ===
500 multiplied by 10 and then subtracting 1000 results in 4000.


## RAG QueryEngine Tools

In [29]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [30]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [23]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [24]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [25]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [26]:
obj_index = ObjectIndex.from_objects(
    query_engine_tools,
    index_cls=VectorStoreIndex,
)

In [27]:
llm = OpenAI(model="gpt-3.5-turbo")
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=1),
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=True,
)
agent = AgentRunner(agent_worker)

In [28]:
response = agent.chat(
    "What is the revenue of Uber in 2021?"
)

Added user message to memory: What is the revenue of Uber in 2021?
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What is the revenue of Uber in 2021?"}
=== Function Output ===
Encountered error: shapes (1536,) and (384,) not aligned: 1536 (dim 0) != 384 (dim 0)
=== LLM Response ===
It seems there was an error in retrieving the revenue information for Uber in 2021. Let me try to fetch the information again.
=== Calling Function ===
Calling function: uber_10k with args: {"input": "revenue of Uber in 2021"}
=== Function Output ===
Encountered error: shapes (1536,) and (384,) not aligned: 1536 (dim 0) != 384 (dim 0)
=== LLM Response ===
I apologize for the inconvenience. It seems there is still an issue with retrieving the revenue information for Uber in 2021. Let me attempt to gather the information using a different approach.
=== Calling Function ===
Calling function: uber_10k with args: {"input": "revenue of Uber in 2021"}
=== Function Output ===
Encountered 